In [1]:
import copy

import numpy as np
import pandas as pd
from collections import Counter, OrderedDict, defaultdict
#from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB #if we'll decide to test different classifiers
#from sklearn.svm import SVC, NuSVC, LinearSVC #if we'll decide to test different classifiers
from sklearn.datasets import load_svmlight_file
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from math import floor
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import ipdb

import tqdm

# for Kseniia's model
from cotrain import SSLTrainSet
from cotrain import BinaryCoTrainer
from dataloaders.dataloader import *

import warnings
warnings.simplefilter('ignore')

%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'
plt.style.use('seaborn-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

#увеличим дефолтный размер графиков
from pylab import rcParams
import pandas as pd
import numpy as np

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
import plotly.graph_objs as go

import tqdm

/usr/local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
df = pd.DataFrame(columns=['Algorithm', 'Dataset', 'L_Size', 'U_Size', 'T_Size', 'Accuracy'])
print(df)

Empty DataFrame
Columns: [Algorithm, Dataset, L_Size, U_Size, T_Size, Accuracy]
Index: []


In [3]:
all_datasets = [
    "pendigits", "mnist", "mnist_fashion", "20_newsgroups", "SMS_spam_ham" , "soccer", "mushroom"]

splits = [[30, 988 , 150], [60, 958, 150], [110, 908, 150], [195, 823, 150],
         [235, 783 , 150], [265, 753, 150], [340, 678, 150], [360, 658, 150], [380, 638, 150], [420, 598, 150]]

bin_dict = dict.fromkeys(all_datasets)

bin_dict["pendigits"] = [[4, 9]]
bin_dict["mnist"] = [[4, 9]]
bin_dict["mnist_fashion"] = [[0, 1]]
bin_dict["20_newsgroups"] = [[3, 4], [9,1]]
bin_dict["SMS_spam_ham"] = [[0, 1]]
bin_dict["soccer"] = [[0, 1]]
bin_dict["mushroom"] = [[0, 1]]

random_states = list(range(0, 101, 10))

rf_params = {'n_estimators': 50}

In [4]:
for dataset in all_datasets:
    for binary in bin_dict.get(dataset):
        for split in splits:
            accuracy_list = np.zeros(len(random_states)) 
            accuracy_list_cp = np.zeros(len(random_states))
            for idx, random_state in enumerate(random_states):
                print("\nDataset : " + dataset," Binary : " + str(binary), " Split : " +str(split))
                X1, Y1, X1u, X2, Y2, X2u, Xtest,Ytest  = get_samples(dataset=dataset,
                                                                     split=split,
                                                                     random_state=random_state,
                                                                     binary=binary,
                                                                     model='co-train')

                rf_params['random_state'] = random_state

                co_train = BinaryCoTrainer([RandomForestClassifier(**rf_params) for _ in range(2)])

                train_sets = [SSLTrainSet(X1, Y1, X1u), SSLTrainSet(X2, Y2, X2u)]
                co_train.fit(train_sets)

                co_scores = []
                for pred in co_train.predict(Xtest):
                    co_scores.append(accuracy_score(pred, Ytest))

                accuracy_list[idx] = np.mean(co_scores)

                #baseline
                model_bl1 = RandomForestClassifier(**rf_params)
                model_bl2 = RandomForestClassifier(**rf_params)        

                model_bl1.fit(X1, Y1)
                model_bl2.fit(X2, Y2)

                model_bl1_accuracy = accuracy_score(model_bl1.predict(Xtest), Ytest)
                model_bl2_accuracy = accuracy_score(model_bl2.predict(Xtest), Ytest)
                
                accuracy_list_cp[idx] = np.mean((model_bl1_accuracy,model_bl2_accuracy))                                
            df = df.append({'Algorithm':'co-train', 'Dataset': dataset, 'L_Size' : split[0], 'U_Size' : split[1],'T_Size' : split[2], 'Accuracy' : np.mean(accuracy_list)}, ignore_index=True)
            df = df.append({'Algorithm':'co-train baseline', 'Dataset': dataset, 'L_Size' : split[0], 'U_Size' : split[1],'T_Size' : split[2], 'Accuracy' : np.mean(accuracy_list_cp)}, ignore_index=True)        


Dataset : pendigits  Binary : [4, 9]  Split : [30, 988, 150]
Size of labeled data (15, 16)
Size of unlabeled data (494, 16)
Size of tets data (150, 16)

Dataset : pendigits  Binary : [4, 9]  Split : [30, 988, 150]
Size of labeled data (15, 16)
Size of unlabeled data (494, 16)
Size of tets data (150, 16)

Dataset : pendigits  Binary : [4, 9]  Split : [30, 988, 150]
Size of labeled data (15, 16)
Size of unlabeled data (494, 16)
Size of tets data (150, 16)

Dataset : pendigits  Binary : [4, 9]  Split : [30, 988, 150]
Size of labeled data (15, 16)
Size of unlabeled data (494, 16)
Size of tets data (150, 16)

Dataset : pendigits  Binary : [4, 9]  Split : [30, 988, 150]
Size of labeled data (15, 16)
Size of unlabeled data (494, 16)
Size of tets data (150, 16)

Dataset : pendigits  Binary : [4, 9]  Split : [30, 988, 150]
Size of labeled data (15, 16)
Size of unlabeled data (494, 16)
Size of tets data (150, 16)

Dataset : pendigits  Binary : [4, 9]  Split : [30, 988, 150]
Size of labeled data


Dataset : pendigits  Binary : [4, 9]  Split : [235, 783, 150]
Size of labeled data (117, 16)
Size of unlabeled data (392, 16)
Size of tets data (150, 16)

Dataset : pendigits  Binary : [4, 9]  Split : [265, 753, 150]
Size of labeled data (132, 16)
Size of unlabeled data (377, 16)
Size of tets data (150, 16)

Dataset : pendigits  Binary : [4, 9]  Split : [265, 753, 150]
Size of labeled data (132, 16)
Size of unlabeled data (377, 16)
Size of tets data (150, 16)

Dataset : pendigits  Binary : [4, 9]  Split : [265, 753, 150]
Size of labeled data (132, 16)
Size of unlabeled data (377, 16)
Size of tets data (150, 16)

Dataset : pendigits  Binary : [4, 9]  Split : [265, 753, 150]
Size of labeled data (132, 16)
Size of unlabeled data (377, 16)
Size of tets data (150, 16)

Dataset : pendigits  Binary : [4, 9]  Split : [265, 753, 150]
Size of labeled data (132, 16)
Size of unlabeled data (377, 16)
Size of tets data (150, 16)

Dataset : pendigits  Binary : [4, 9]  Split : [265, 753, 150]
Size of


Dataset : pendigits  Binary : [4, 9]  Split : [420, 598, 150]
Size of labeled data (210, 16)
Size of unlabeled data (299, 16)
Size of tets data (150, 16)

Dataset : pendigits  Binary : [4, 9]  Split : [420, 598, 150]
Size of labeled data (210, 16)
Size of unlabeled data (299, 16)
Size of tets data (150, 16)

Dataset : pendigits  Binary : [4, 9]  Split : [420, 598, 150]
Size of labeled data (210, 16)
Size of unlabeled data (299, 16)
Size of tets data (150, 16)

Dataset : mnist  Binary : [4, 9]  Split : [30, 988, 150]
Size of labeled data (15, 784)
Size of unlabeled data (494, 784)
Size of tets data (150, 784)

Dataset : mnist  Binary : [4, 9]  Split : [30, 988, 150]
Size of labeled data (15, 784)
Size of unlabeled data (494, 784)
Size of tets data (150, 784)

Dataset : mnist  Binary : [4, 9]  Split : [30, 988, 150]
Size of labeled data (15, 784)
Size of unlabeled data (494, 784)
Size of tets data (150, 784)

Dataset : mnist  Binary : [4, 9]  Split : [30, 988, 150]
Size of labeled data 


Dataset : mnist  Binary : [4, 9]  Split : [235, 783, 150]
Size of labeled data (117, 784)
Size of unlabeled data (392, 784)
Size of tets data (150, 784)

Dataset : mnist  Binary : [4, 9]  Split : [235, 783, 150]
Size of labeled data (117, 784)
Size of unlabeled data (392, 784)
Size of tets data (150, 784)

Dataset : mnist  Binary : [4, 9]  Split : [235, 783, 150]
Size of labeled data (117, 784)
Size of unlabeled data (392, 784)
Size of tets data (150, 784)

Dataset : mnist  Binary : [4, 9]  Split : [235, 783, 150]
Size of labeled data (117, 784)
Size of unlabeled data (392, 784)
Size of tets data (150, 784)

Dataset : mnist  Binary : [4, 9]  Split : [265, 753, 150]
Size of labeled data (132, 784)
Size of unlabeled data (377, 784)
Size of tets data (150, 784)

Dataset : mnist  Binary : [4, 9]  Split : [265, 753, 150]
Size of labeled data (132, 784)
Size of unlabeled data (377, 784)
Size of tets data (150, 784)

Dataset : mnist  Binary : [4, 9]  Split : [265, 753, 150]
Size of labeled d


Dataset : mnist  Binary : [4, 9]  Split : [420, 598, 150]
Size of labeled data (210, 784)
Size of unlabeled data (299, 784)
Size of tets data (150, 784)

Dataset : mnist  Binary : [4, 9]  Split : [420, 598, 150]
Size of labeled data (210, 784)
Size of unlabeled data (299, 784)
Size of tets data (150, 784)

Dataset : mnist  Binary : [4, 9]  Split : [420, 598, 150]
Size of labeled data (210, 784)
Size of unlabeled data (299, 784)
Size of tets data (150, 784)

Dataset : mnist  Binary : [4, 9]  Split : [420, 598, 150]
Size of labeled data (210, 784)
Size of unlabeled data (299, 784)
Size of tets data (150, 784)

Dataset : mnist  Binary : [4, 9]  Split : [420, 598, 150]
Size of labeled data (210, 784)
Size of unlabeled data (299, 784)
Size of tets data (150, 784)

Dataset : mnist_fashion  Binary : [0, 1]  Split : [30, 988, 150]
Size of labeled data (15, 784)
Size of unlabeled data (494, 784)
Size of tets data (150, 784)

Dataset : mnist_fashion  Binary : [0, 1]  Split : [30, 988, 150]
Size

...


Dataset : SMS_spam_ham  Binary : [0, 1]  Split : [340, 678, 150]
Size of labeled data (170, 1892)
Size of unlabeled data (339, 1892)
Size of tets data (150, 1892)

Dataset : SMS_spam_ham  Binary : [0, 1]  Split : [340, 678, 150]
Size of labeled data (170, 1892)
Size of unlabeled data (339, 1892)
Size of tets data (150, 1892)

Dataset : SMS_spam_ham  Binary : [0, 1]  Split : [340, 678, 150]
Size of labeled data (170, 1892)
Size of unlabeled data (339, 1892)
Size of tets data (150, 1892)

Dataset : SMS_spam_ham  Binary : [0, 1]  Split : [340, 678, 150]
Size of labeled data (170, 1892)
Size of unlabeled data (339, 1892)
Size of tets data (150, 1892)

Dataset : SMS_spam_ham  Binary : [0, 1]  Split : [340, 678, 150]
Size of labeled data (170, 1892)
Size of unlabeled data (339, 1892)
Size of tets data (150, 1892)

Dataset : SMS_spam_ham  Binary : [0, 1]  Split : [340, 678, 150]
Size of labeled data (170, 1892)
Size of unlabeled data (339, 1892)
Size of tets data (150, 1892)

Dataset : SMS_s


Dataset : soccer  Binary : [0, 1]  Split : [60, 958, 150]
Size of labeled data (30, 7)
Size of unlabeled data (479, 7)
Size of tets data (150, 7)

Dataset : soccer  Binary : [0, 1]  Split : [60, 958, 150]
Size of labeled data (30, 7)
Size of unlabeled data (479, 7)
Size of tets data (150, 7)

Dataset : soccer  Binary : [0, 1]  Split : [60, 958, 150]
Size of labeled data (30, 7)
Size of unlabeled data (479, 7)
Size of tets data (150, 7)

Dataset : soccer  Binary : [0, 1]  Split : [60, 958, 150]
Size of labeled data (30, 7)
Size of unlabeled data (479, 7)
Size of tets data (150, 7)

Dataset : soccer  Binary : [0, 1]  Split : [60, 958, 150]
Size of labeled data (30, 7)
Size of unlabeled data (479, 7)
Size of tets data (150, 7)

Dataset : soccer  Binary : [0, 1]  Split : [60, 958, 150]
Size of labeled data (30, 7)
Size of unlabeled data (479, 7)
Size of tets data (150, 7)

Dataset : soccer  Binary : [0, 1]  Split : [60, 958, 150]
Size of labeled data (30, 7)
Size of unlabeled data (479, 7


Dataset : soccer  Binary : [0, 1]  Split : [340, 678, 150]
Size of labeled data (170, 7)
Size of unlabeled data (339, 7)
Size of tets data (150, 7)

Dataset : soccer  Binary : [0, 1]  Split : [340, 678, 150]
Size of labeled data (170, 7)
Size of unlabeled data (339, 7)
Size of tets data (150, 7)

Dataset : soccer  Binary : [0, 1]  Split : [340, 678, 150]
Size of labeled data (170, 7)
Size of unlabeled data (339, 7)
Size of tets data (150, 7)

Dataset : soccer  Binary : [0, 1]  Split : [340, 678, 150]
Size of labeled data (170, 7)
Size of unlabeled data (339, 7)
Size of tets data (150, 7)

Dataset : soccer  Binary : [0, 1]  Split : [340, 678, 150]
Size of labeled data (170, 7)
Size of unlabeled data (339, 7)
Size of tets data (150, 7)

Dataset : soccer  Binary : [0, 1]  Split : [340, 678, 150]
Size of labeled data (170, 7)
Size of unlabeled data (339, 7)
Size of tets data (150, 7)

Dataset : soccer  Binary : [0, 1]  Split : [340, 678, 150]
Size of labeled data (170, 7)
Size of unlabele


Dataset : mushroom  Binary : [0, 1]  Split : [60, 958, 150]
Size of labeled data (30, 22)
Size of unlabeled data (479, 22)
Size of tets data (150, 22)

Dataset : mushroom  Binary : [0, 1]  Split : [60, 958, 150]
Size of labeled data (30, 22)
Size of unlabeled data (479, 22)
Size of tets data (150, 22)

Dataset : mushroom  Binary : [0, 1]  Split : [60, 958, 150]
Size of labeled data (30, 22)
Size of unlabeled data (479, 22)
Size of tets data (150, 22)

Dataset : mushroom  Binary : [0, 1]  Split : [60, 958, 150]
Size of labeled data (30, 22)
Size of unlabeled data (479, 22)
Size of tets data (150, 22)

Dataset : mushroom  Binary : [0, 1]  Split : [60, 958, 150]
Size of labeled data (30, 22)
Size of unlabeled data (479, 22)
Size of tets data (150, 22)

Dataset : mushroom  Binary : [0, 1]  Split : [60, 958, 150]
Size of labeled data (30, 22)
Size of unlabeled data (479, 22)
Size of tets data (150, 22)

Dataset : mushroom  Binary : [0, 1]  Split : [60, 958, 150]
Size of labeled data (30, 2


Dataset : mushroom  Binary : [0, 1]  Split : [340, 678, 150]
Size of labeled data (170, 22)
Size of unlabeled data (339, 22)
Size of tets data (150, 22)

Dataset : mushroom  Binary : [0, 1]  Split : [340, 678, 150]
Size of labeled data (170, 22)
Size of unlabeled data (339, 22)
Size of tets data (150, 22)

Dataset : mushroom  Binary : [0, 1]  Split : [340, 678, 150]
Size of labeled data (170, 22)
Size of unlabeled data (339, 22)
Size of tets data (150, 22)

Dataset : mushroom  Binary : [0, 1]  Split : [340, 678, 150]
Size of labeled data (170, 22)
Size of unlabeled data (339, 22)
Size of tets data (150, 22)

Dataset : mushroom  Binary : [0, 1]  Split : [340, 678, 150]
Size of labeled data (170, 22)
Size of unlabeled data (339, 22)
Size of tets data (150, 22)

Dataset : mushroom  Binary : [0, 1]  Split : [340, 678, 150]
Size of labeled data (170, 22)
Size of unlabeled data (339, 22)
Size of tets data (150, 22)

Dataset : mushroom  Binary : [0, 1]  Split : [340, 678, 150]
Size of labele

In [5]:
df.to_csv('results/MYALGO_TEST2.csv')